In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# --------- User inputs (edit if needed) ----------
acled_path = "ACLED_May_09_25_Gaza.xlsx"
hosp_path  = "Hospitals_OpenCloseoverTime.xlsx"

date_ranges = [
    (datetime(2023, 10, 7), datetime(2023, 11, 3)),
    (datetime(2023, 12, 11), datetime(2024, 4, 28)),
    (datetime(2024, 11, 11), datetime(2025, 2, 2)),
]

# This dictionary maps the final wide-column labels (keys) to the list of ACLED sub-event values that
# should count toward that column. Adjust as needed.
attack_categories = {
    'Air/drone strike': ['Air/drone strike'],
    'Ground Attacks': ['Armed clash', 'Attack', 'Remote explosive/landmine/IED', 'Grenade', 'Change to group/activity'],
    'Shelling/artillery/missile attack': ['Shelling/artillery/missile attack'],
    'Civil Unrest': ['Mob violence', 'Looting/property destruction', 'Violent demonstration', 'Peaceful protest',
                     'Disrupted weapons use', 'Excessive force against protesters', 'Arrests',
                     'Protest with intervention', 'Abduction/forced disappearance', 'Sexual violence'],
    'Other': ['Other']
}

# adjustable bin length (2 weeks by default)
BIN_LENGTH = timedelta(days=14)

# column detection candidates (edit if you want to hardcode names)
ACLED_DATE_COLS = ['event_date', 'date', 'Event Date', 'event_date_ymd']
ACLED_LAT_COLS = ['latitude', 'Latitude', 'LAT']
ACLED_LON_COLS = ['longitude', 'Longitude', 'LON']
SUB_EVENT_COLS = ['sub_event_type', 'Sub Event Type', 'sub-event_type', 'sub_event', 'sub_eventtype']

HOSP_LAT_COLS = ['Latitude (Y)', 'Latitude', 'latitude', 'LAT']
HOSP_LON_COLS = ['Longitude (X)', 'Longitude', 'longitude', 'LON']

output_folder = "."  # change if you'd like a different output location
# -------------------------------------------------

def find_col(df, candidates):
    cols_lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        key = cand.lower()
        if key in cols_lower:
            return cols_lower[key]
    for c in df.columns:
        if c.replace(" ", "").lower() in [x.replace(" ", "").lower() for x in candidates]:
            return c
    return None

def haversine_np(lon1, lat1, lon2, lat2):
    # all inputs are numpy arrays (can be 1D or 2D), function returns distances in km
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0 * c
    return km

def map_sub_to_category_map(attack_categories):
    m = {}
    for cat, subs in attack_categories.items():
        for s in subs:
            m[s.strip().lower()] = cat
    return m

def build_bin_edges(start_dt, end_dt, bin_length):
    edges = []
    cur = pd.Timestamp(start_dt)
    # create edges inclusive of start; we'll add final edge after loop to make end inclusive
    while cur <= pd.Timestamp(end_dt):
        edges.append(cur)
        cur = cur + bin_length
    # final edge is end_dt + 1 day so pd.cut with right=False will include events on end_dt
    edges.append(pd.Timestamp(end_dt) + pd.Timedelta(days=1))
    return edges

def ensure_all_categories(df, categories):
    """Ensure df has columns for every category in categories; fill missing with zeros."""
    for c in categories:
        if c not in df.columns:
            df[c] = 0
    return df

def main():
    os.makedirs(output_folder, exist_ok=True)
    print("Loading files...")
    acled = pd.read_excel(acled_path, engine='openpyxl')
    hospitals = pd.read_excel(hosp_path, engine='openpyxl')

    date_col = find_col(acled, ACLED_DATE_COLS)
    lat_col = find_col(acled, ACLED_LAT_COLS)
    lon_col = find_col(acled, ACLED_LON_COLS)
    sub_event_col = find_col(acled, SUB_EVENT_COLS)

    hosp_lat = find_col(hospitals, HOSP_LAT_COLS)
    hosp_lon = find_col(hospitals, HOSP_LON_COLS)

    if date_col is None:
        raise ValueError("Could not find a date column in ACLED. Candidates tried: " + str(ACLED_DATE_COLS))
    if lat_col is None or lon_col is None:
        raise ValueError("Could not find latitude/longitude columns in ACLED. Candidates tried: " + str(ACLED_LAT_COLS) + ", " + str(ACLED_LON_COLS))
    if hosp_lat is None or hosp_lon is None:
        raise ValueError("Could not find Latitude/Longitude in hospitals file. Candidates tried: " + str(HOSP_LAT_COLS) + ", " + str(HOSP_LON_COLS))
    if sub_event_col is None:
        if 'sub_event_type' in acled.columns:
            sub_event_col = 'sub_event_type'
        else:
            raise ValueError("Sub-event column not found. Ensure there is a sub-event column (e.g., 'sub_event_type').")

    print(f"Using date column: {date_col}, lat/lon: {lat_col}/{lon_col}, sub-event column: {sub_event_col}")
    print(f"Hospital lat/lon columns: {hosp_lat}/{hosp_lon}")

    # Standardize types
    acled[date_col] = pd.to_datetime(acled[date_col], errors='coerce')
    acled = acled.dropna(subset=[date_col])
    acled[lat_col] = pd.to_numeric(acled[lat_col], errors='coerce')
    acled[lon_col] = pd.to_numeric(acled[lon_col], errors='coerce')
    hospitals[hosp_lat] = pd.to_numeric(hospitals[hosp_lat], errors='coerce')
    hospitals[hosp_lon] = pd.to_numeric(hospitals[hosp_lon], errors='coerce')

    acled = acled.dropna(subset=[lat_col, lon_col]).reset_index(drop=True)
    hospitals = hospitals.dropna(subset=[hosp_lat, hosp_lon]).reset_index(drop=True)

    hosp_lats = hospitals[hosp_lat].to_numpy()
    hosp_lons = hospitals[hosp_lon].to_numpy()
    hosp_name_col = find_col(hospitals, ['name', 'Name', 'Hospital Name', 'hospital'])

    map_sub_to_cat = map_sub_to_category_map(attack_categories)
    default_category = 'Other'

    # container to collect all period binned pivots
    combined_rows = []

    for i, (start_dt, end_dt) in enumerate(date_ranges, start=1):
        print("\n" + "="*60)
        print(f"Processing date range #{i}: {start_dt.date()} to {end_dt.date()}")
        mask = (acled[date_col] >= pd.Timestamp(start_dt)) & (acled[date_col] <= pd.Timestamp(end_dt))
        acled_period = acled.loc[mask].copy().reset_index(drop=True)
        print(f"Found {len(acled_period)} events in this period.")

        if len(acled_period) == 0:
            # still build empty pivot with bins and zeros
            event_present = False
        else:
            event_present = True
            # compute nearest hospital for each event
            ev_lats = acled_period[lat_col].to_numpy()[:, None]   # shape (n,1)
            ev_lons = acled_period[lon_col].to_numpy()[:, None]   # shape (n,1)
            # hosp arrays shapes (m,) -> broadcast to (n,m) in haversine
            dists_km = haversine_np(ev_lons, ev_lats, hosp_lons[None, :], hosp_lats[None, :])
            nearest_idx = np.argmin(dists_km, axis=1)
            nearest_dist = dists_km[np.arange(dists_km.shape[0]), nearest_idx]

            acled_period['nearest_hospital_index'] = nearest_idx
            if hosp_name_col:
                # use iloc because nearest_idx are integer positions
                acled_period['nearest_hospital_name'] = hospitals.iloc[nearest_idx][hosp_name_col].values
            else:
                acled_period['nearest_hospital_name'] = hospitals.index[nearest_idx].astype(str)
            acled_period['nearest_hospital_distance_km'] = nearest_dist

            def map_sub(sub):
                if pd.isna(sub):
                    return default_category
                return map_sub_to_cat.get(str(sub).strip().lower(), default_category)
            acled_period['attack_category'] = acled_period[sub_event_col].apply(map_sub)

        # Build bins: edges and human-friendly labels
        bin_edges = build_bin_edges(start_dt, end_dt, BIN_LENGTH)
        bin_labels = []
        bin_starts = []
        bin_ends = []
        for b_start, b_end in zip(bin_edges[:-1], bin_edges[1:]):
            # label shows inclusive start to inclusive end (we subtracted 1 day from b_end)
            label = f"{b_start.date()} to {(b_end - pd.Timedelta(days=1)).date()}"
            bin_labels.append(label)
            bin_starts.append(b_start.date())
            bin_ends.append((b_end - pd.Timedelta(days=1)).date())

        # Create a small dataframe with bin metadata to merge later so each row gets correct start/end
        bins_df = pd.DataFrame({
            'period': bin_labels,
            'bin_start': [d.isoformat() for d in bin_starts],
            'bin_end': [d.isoformat() for d in bin_ends]
        })

        if event_present:
            # cut into bins (right=False so bins are [start, end) and our final edge included via +1day)
            acled_period['time_bin'] = pd.cut(acled_period[date_col],
                                              bins=bin_edges,
                                              labels=bin_labels,
                                              right=False)
            counts = (acled_period
                      .groupby(['time_bin', 'attack_category'])
                      .size()
                      .reset_index(name='count'))
            pivot = counts.pivot(index='time_bin', columns='attack_category', values='count').fillna(0)
        else:
            # create empty pivot with bins as index and categories as columns
            pivot = pd.DataFrame(0, index=pd.Index(bin_labels, name='time_bin'),
                                 columns=list(attack_categories.keys()))

        # make sure every category column exists
        pivot = ensure_all_categories(pivot, list(attack_categories.keys()))

        # Reset index and rename the bin column to 'period' so it matches the screenshot
        pivot = pivot.reset_index().rename(columns={'time_bin': 'period'})

        # Merge bin-level starts/ends so each row's period_start/period_end match the 'period' text
        pivot = pd.merge(bins_df, pivot, on='period', how='right', sort=False)
        # If merge produced NaNs for bin_start/bin_end (shouldn't), fill with overall range start/end
        pivot['bin_start'] = pivot['bin_start'].fillna(pd.Timestamp(start_dt).date().isoformat())
        pivot['bin_end']   = pivot['bin_end'].fillna(pd.Timestamp(end_dt).date().isoformat())

        # Add period metadata column at front: period_id (same for all rows belonging to this date-range)
        pivot.insert(0, 'period_id', i)

        # rename bin_start/bin_end to desired column names
        pivot = pivot.rename(columns={'bin_start': 'period_start', 'bin_end': 'period_end'})

        # Ensure category columns are ints and in a stable order (use attack_categories order or override)
        desired_category_order = list(attack_categories.keys())  # <-- change this list if you want a different column order
        for c in desired_category_order:
            if c not in pivot.columns:
                pivot[c] = 0
            # fill NaNs then cast to int
            pivot[c] = pivot[c].fillna(0).astype(int)

        # Reorder columns so the table matches the screenshot and has period_start/period_end matching the period
        ordered_cols = ['period_id', 'period_start', 'period_end', 'period'] + desired_category_order
        # add any unexpected extra cols at the end (defensive)
        extra_cols = [c for c in pivot.columns if c not in ordered_cols]
        pivot = pivot[[*ordered_cols, *extra_cols]]

        # store for combined table
        combined_rows.append(pivot)


    # after processing all periods, concatenate combined_rows into a single DataFrame
    if len(combined_rows) > 0:
        combined_df = pd.concat(combined_rows, ignore_index=True, sort=False)

        # final column order for combined DF
        cols = ['period_id', 'period_start', 'period_end', 'period'] + list(attack_categories.keys())
        # add any missing columns (defensive)
        for c in cols:
            if c not in combined_df.columns:
                combined_df[c] = 0
        # ensure categories are ints
        for c in attack_categories.keys():
            combined_df[c] = combined_df[c].fillna(0).astype(int)

        combined_df = combined_df[cols]
    else:
        # empty combined table with expected columns
        combined_df = pd.DataFrame(columns=['period_id', 'period_start', 'period_end', 'period'] + list(attack_categories.keys()))

    combined_path = os.path.join(output_folder, "injuries.xlsx")
    with pd.ExcelWriter(combined_path, engine='openpyxl') as writer:
        combined_df.to_excel(writer, sheet_name='all_binned_counts', index=False)

    print(f"\nSaved combined binned counts to: {combined_path}")
    print("Done.")

if __name__ == "__main__":
    main()


Loading files...
Using date column: event_date, lat/lon: latitude/longitude, sub-event column: sub_event_type
Hospital lat/lon columns: Latitude (Y)/Longitude (X)

Processing date range #1: 2023-10-07 to 2023-11-03
Found 1107 events in this period.

Processing date range #2: 2023-12-11 to 2024-04-28
Found 5517 events in this period.

Processing date range #3: 2024-11-11 to 2025-02-02
Found 2163 events in this period.

Saved combined binned counts to: .\injuries.xlsx
Done.


C:\Users\ctr22\AppData\Local\Temp\ipykernel_8280\77958883.py:201: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['time_bin', 'attack_category'])
C:\Users\ctr22\AppData\Local\Temp\ipykernel_8280\77958883.py:201: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['time_bin', 'attack_category'])
C:\Users\ctr22\AppData\Local\Temp\ipykernel_8280\77958883.py:201: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .gro